In [1]:

!apt-get update
!apt-get install -y build-essential


!curl -LsSf https://astral.sh/uv/install.sh | python -


!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118


!pip install axolotl
!pip install datasets transformers peft bitsandbytes
!pip install huggingface-hub

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 https://cli.github.com/packages stable InRelease [3,917 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [3,532 kB]
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages

KeyboardInterrupt: 

In [1]:
from huggingface_hub import login
login()

In [3]:
from google.colab import files


!mkdir -p data

!wget https://huggingface.co/datasets/demoversion/cf-cpp-to-python-code-generation/resolve/main/train_openai_response_transformed.jsonl -O ./data/train_openai_response_transformed.jsonl

!wget https://huggingface.co/datasets/demoversion/cf-cpp-to-python-code-generation/resolve/main/val_openai_response_transformed.jsonl -O ./data/val_openai_response_transformed.jsonl

--2025-11-19 02:08:17--  https://huggingface.co/datasets/demoversion/cf-cpp-to-python-code-generation/resolve/main/train_openai_response_transformed.jsonl
Resolving huggingface.co (huggingface.co)... 18.239.50.16, 18.239.50.103, 18.239.50.49, ...
Connecting to huggingface.co (huggingface.co)|18.239.50.16|:443... connected.
HTTP request sent, awaiting response... 307 Temporary Redirect
Location: /api/resolve-cache/datasets/demoversion/cf-cpp-to-python-code-generation/23ce116110db54eee84ddd8180ce1514812844c8/train_openai_response_transformed.jsonl?%2Fdatasets%2Fdemoversion%2Fcf-cpp-to-python-code-generation%2Fresolve%2Fmain%2Ftrain_openai_response_transformed.jsonl=&etag=%22387cf31722231549f86fe171e13dfffcc13e8e0d%22 [following]
--2025-11-19 02:08:17--  https://huggingface.co/api/resolve-cache/datasets/demoversion/cf-cpp-to-python-code-generation/23ce116110db54eee84ddd8180ce1514812844c8/train_openai_response_transformed.jsonl?%2Fdatasets%2Fdemoversion%2Fcf-cpp-to-python-code-generation%2

In [31]:
%%writefile config.yaml

base_model: meta-llama/Llama-3.2-3B-Instruct
model_type: llama
tokenizer_type: LlamaTokenizerFast
tokenizer_config: meta-llama/Llama-3.2-3B-Instruct

datasets:
  - path: ./data/train_openai_response_transformed.jsonl
    type: chat_template

val_set_size: 0.1
val_file: ./data/val_openai_response_transformed.jsonl

output_dir: ./outputs
num_epochs: 3
micro_batch_size: 2
gradient_accumulation_steps: 4
learning_rate: 3e-4
warmup_steps: 100
logging_steps: 10
save_steps: 100
eval_steps: 100
max_seq_length: 1024
seed: 42

load_in_8bit: true
fp16: true
bf16: false
gradient_checkpointing: true
optimizer: adamw_torch_fused

adapter: lora
lora_r: 8
lora_alpha: 16
lora_dropout: 0.05
lora_target_modules:
  - q_proj
  - k_proj
  - v_proj
  - o_proj


save_safetensors: true
type_of_model: llama
strict: false
shuffle_merged_datasets: true
train_on_inputs: false
torch_dtype: torch.float16
world_size: 1




Writing config.yaml


In [ ]:
!pip uninstall -y torch torchvision torchaudio transformers peft

!pip install torch==2.1.2 torchvision==0.16.1 torchaudio==2.1.2 --index-url https://download.pytorch.org/whl/cu121

!pip install transformers==4.37.2 peft==0.10.0 accelerate==0.26.0 bitsandbytes==0.42.0

!pip install axolotl


In [32]:
!mkdir -p config
!mv config.yaml config/llama-3.2-3b-lora.yml

In [33]:
!cd /content && python -m axolotl.cli.preprocess config/llama-3.2-3b-lora.yml


2025-11-19 02:35:59.771018: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763519759.805493   11269 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763519759.816344   11269 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1763519759.841399   11269 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1763519759.841445   11269 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1763519759.841453   11269 computation_placer.cc:177] computation placer alr

In [35]:
from datasets import load_from_disk

ds = load_from_disk("./last_run_prepared/0d8b032105993cd50c291ef141081e49")
print(ds[0])


{'input_ids': [128000, 128006, 9125, 128007, 271, 38766, 1303, 33025, 2696, 25, 6790, 220, 2366, 18, 198, 15724, 2696, 25, 220, 777, 4723, 220, 2366, 20, 271, 2675, 527, 459, 8479, 430, 27983, 13325, 2082, 3196, 389, 279, 3984, 356, 1044, 2592, 2082, 627, 5451, 10552, 279, 2082, 11, 1243, 7068, 279, 13890, 13325, 2082, 627, 7927, 2082, 1288, 1373, 505, 5410, 1988, 323, 3350, 311, 5410, 2612, 627, 2746, 1070, 374, 2587, 12496, 304, 279, 356, 1044, 2082, 311, 1373, 1988, 477, 3350, 2612, 505, 3626, 304, 2254, 11, 40071, 433, 627, 2028, 2082, 374, 459, 8292, 315, 264, 15022, 15840, 3575, 11, 3234, 539, 1304, 904, 32946, 922, 279, 1988, 3645, 6062, 433, 690, 3775, 627, 19648, 279, 8066, 13325, 2082, 4871, 264, 2082, 2565, 449, 24657, 1203, 36178, 627, 13617, 512, 74694, 12958, 198, 475, 5826, 271, 755, 11886, 4019, 262, 1522, 271, 333, 1328, 609, 565, 624, 13568, 3902, 21762, 262, 11886, 746, 74694, 128009, 128006, 882, 128007, 271, 14196, 4077, 1085, 366, 9862, 397, 1085, 366, 3295, 397, 

In [36]:
tok = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-3B-Instruct")
decoded = tok.decode(ds[0]["input_ids"])
print(decoded)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 19 Nov 2025

You are an agent that generates Python code based on the provided C++ source code.
First explain the code, then generate the equivalent Python code.
Your code should read from standard input and write to standard output.
If there is custom logic in the C++ code to read input or write output from files in local, Ignore it.
This code is an implementation of a competitive programming problem, Do not make any assumptions about the input format otherwise it will fail.
Put the generated Python code inside a code block with triple backticks.
Example:
```python
import sys

def solve():
    pass

if __name__ == "__main__":
    solve()
```<|eot_id|><|start_header_id|>user<|end_header_id|>

```
#include <iostream>
#include <vector>
#include <map>
#include <set>
#include <algorithm>
#include <sstream>
#include <fstream>
#include <cmath>
#include <stack>
#include <cstdio>
#inc

In [37]:
!cd /content && python -m axolotl.cli.train config/llama-3.2-3b-lora.yml

2025-11-19 02:49:47.836177: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763520587.868844   14831 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763520587.878838   14831 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1763520587.903639   14831 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1763520587.903701   14831 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1763520587.903711   14831 computation_placer.cc:177] computation placer alr

In [38]:

!nvidia-smi

import os
output_files = os.listdir('./outputs')
print("Output files:", output_files)

Wed Nov 19 02:55:59 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   66C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-3.2-3B-Instruct",
    load_in_8bit=True,
    device_map="auto"
)

model = PeftModel.from_pretrained(
    model,
    "./outputs",
    device_map="auto"
)




# Push to HuggingFace
# merged_model.push_to_hub("your-username/llama-3.2-3b-finetuned")
# AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-3B-Instruct").push_to_hub("your-username/llama-3.2-3b-finetuned")

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 752.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 278.12 MiB is free. Process 233674 has 14.47 GiB memory in use. Of the allocated memory 14.26 GiB is allocated by PyTorch, and 98.83 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)